# Premade useful examples

In [ ]:
# Code in this cell enables plotting in the notebook
%matplotlib inline
import spyro

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


Author: Alexandre Olender

Contact: olender@usp.br

This tutorial provides simple examples commonly encountered in seismic imaging model development. These examples serve as a foundation for testing and verifying code implementations before applying them to more complex experiments. You can find these examples within the "examples" folder.

If you are running this notebook in Google colab, please copy the following code into a code block before running the notebook:
```python
# For use in colab only:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

!pip install git+https://github.com/NDF-Poli-USP/spyro.git

## Rectangle example

The Rectangle example is, by default, a 1 km by 1 km rectangle with 0.05 km mesh size and 0.25km absorbing layers. It has a default dictionary located in the rectangles.py file. You can easily modify any isolated dictionary parameter. The example class has a multiple_layer_velocity_model method for quickly adding horizontal velocity layers. For instance, you can create a four-layer experiment with:

In [2]:
Wave_obj = spyro.examples.Rectangle_acoustic()

layer_values = [1.5, 2.0, 2.5, 3.0]
z_switches = [-0.25, -0.5, -0.75]
Wave_obj.multiple_layer_velocity_model(z_switches, layer_values)

Parallelism type: automatic


/home/alexandre/Development/tutorials/spyro-1/spyro/solvers/wave.py:85: UserWarning: No mesh file, Firedrake mesh will be automatically generated.
  warnings.warn(


Let us have a look at the generated model:

In [3]:
spyro.plots.plot_model(Wave_obj, filename="rectangle_model1.png", show=False)

File name rectangle_model1.png


![title](rectangle_model1.png)

You can easily customize this model. Let's create a model with the following specifications:

- Width: 4 km
- Depth: 3 km
- Element size: 100 meters
- No Perfectly Matched Layer (PML)
- Source located 10 meters deep in the middle of the width
- 20 receivers equally spaced between the first and second layers
- 3 layers, equally spaced, with velocities of 1.5 km/s, 2.5 km/s, and 3.5 km/s.

Simply adjust the parameters that deviate from the default values.

In [4]:
dictionary = {}
dictionary["mesh"] = {
    "Lz": 3.0,  # depth in km - always positive
    "Lx": 4.0,  # width in km - always positive
    "h": 0.1,  # mesh size in km
}
dictionary["absorving_boundary_conditions"] = {
    "status": False,
    "pad_length": 0.,
}
dictionary["acquisition"] = {
    "source_locations": [(-0.1, 2.0)],
    "receiver_locations": spyro.create_transect((-1.0, 0.0), (-1.0, 4.0), 20),
}
Wave_obj_rec2 = spyro.examples.Rectangle_acoustic(dictionary=dictionary)
layer_values = [1.5, 2.5, 3.5]
z_switches = [-1.0, -2.0]
Wave_obj_rec2.multiple_layer_velocity_model(z_switches, layer_values)

Parallelism type: automatic


/home/alexandre/Development/tutorials/spyro-1/spyro/solvers/wave.py:85: UserWarning: No mesh file, Firedrake mesh will be automatically generated.
  warnings.warn(


As we can see, you only need to add the parameters that differ from the default values.

In [5]:
spyro.plots.plot_model(Wave_obj_rec2, filename="rectangle_model2.png", show=False)

File name rectangle_model2.png


![title](rectangle_model2.png)

Now, generate your own model based on the Rectangle example with five layers, 6 km width, and 3 km depth.

## Camembert example

A recurring model in the literature for verifying and validating code is the Camembert model, which consists of a higher velocity circle inside an otherwise homogeneous velocity rectangle domain.

Let us create a model with the following specifications:
- 1 km wide,
- 1 km deep,
- 100 meter element size,
- inside circle velocity of 3.5 km/s and 0.2 km radius,
- outside circle velocity of 2.0 km/s,
- 1 ricker source at (-0.1, 0.5) with 6 Hz peak frequency,
- 10 receivers between (-0.9, 0.1) and (-0.9, 0.9).

In [6]:
camembert_dictionary = {}
camembert_dictionary["mesh"] = {
    "Lz": 1.0,  # depth in km - always positive
    "Lx": 1.0,  # width in km - always positive
    "h": 0.05,  # mesh size in km
}
camembert_dictionary["camembert_options"] = {
    "radius": 0.2,
    "outside_velocity": 2.0,
    "inside_circle_velocity": 3.5,
}
camembert_dictionary["acquisition"] = {
    "source_locations": [(-0.1, 0.5)],
    "frequency": 6.0,
    "receiver_locations": spyro.create_transect((-0.9, 0.1), (-0.9, 0.9), 10),
}
camembert_dictionary["visualization"] = {
    "debug_output": True,
}
Wave_obj_queijo_minas = spyro.examples.Camembert_acoustic(dictionary=camembert_dictionary)

Parallelism type: automatic


/home/alexandre/Development/tutorials/spyro-1/spyro/io/dictionaryio.py:301: UserWarning: Both methods of specifying method and cell_type with                     variant used. Method specification taking priority.
  warnings.warn(
/home/alexandre/Development/tutorials/spyro-1/spyro/io/model_parameters.py:610: UserWarning: No velocity model set initially. If using                         user defined conditional or expression, please                             input it in the Wave object.
  warnings.warn(
/home/alexandre/Development/tutorials/spyro-1/spyro/solvers/wave.py:85: UserWarning: No mesh file, Firedrake mesh will be automatically generated.
  warnings.warn(


In [7]:
spyro.plots.plot_model(Wave_obj_queijo_minas, filename="camembert_example.png", show=False)

File name camembert_example.png


![title](camembert_example.png)

Looking at our model, you can see that the circle is not well-defined. The visual plotting capabilities of Firedrake have difficulties with higher order elements, such as the ML4tri we used on this camembert example. To have a more accurate representation of the real velocity model used, you have to use a Paraview version higher or equal to 5.8. The reason we passed a debug output boolean in our dictionary is so that it outputs the velocity model. The figure generated by Paraview can be seen below:

![title](camembert_example_paraview.png)

## Creating your own example

This section is crucial, especially during code development when you need to test or experiment with similar models featuring variations in single or multiple variables. For instance, you may want to train a neural network with changing velocity model files, run the same model in Full Waveform Inversion (FWI) with different receiver setups, or employ FWI while varying inversion options.

Creating a commonly used example can be beneficial for your own use and for other researchers. For example, you may want to test the same velocity model with hundreds of variations of receiver locations.

We can create a default example model such as the one below:

In [ ]:
from spyro.examples.example_model import Example_model_acoustic

default_dictionary = {}
default_dictionary["options"] = {
    "cell_type": "T",  # simplexes such as triangles or tetrahedra (T) or quadrilaterals (Q)
    "variant": "lumped",  # lumped, equispaced or DG, default is lumped
    "degree": 4,  # p order
    "dimension": 2,  # dimension
}
default_dictionary["parallelism"] = {
    "type": "automatic",  # options: automatic (same number of cores for evey processor) or spatial
}
default_dictionary["mesh"] = {
    "Lz": 2.8,  # depth in km - always positive
    "Lx": 6.0,  # width in km - always positive
    "Ly": 0.0,  # thickness in km - always positive
    "mesh_file": "meshes/cut_overthrust.msh",
}
default_dictionary["acquisition"] = {
    "source_type": "ricker",
    "source_locations": [(-0.01, 3.0)],
    "frequency": 5.0,
    "receiver_locations": spyro.create_transect((-0.37, 0.2), (-0.37, 5.8), 300),
}
default_dictionary["absorving_boundary_conditions"] = {
    "status": True,
    "damping_type": "PML",
    "exponent": 2,
    "cmax": 4.5,
    "R": 1e-6,
    "pad_length": 0.75,
}
default_dictionary["synthetic_data"] = {
    "real_velocity_file": "velocity_models/cut_overthrust.hdf5",
}
default_dictionary["time_axis"] = {
    "initial_time": 0.0,  # Initial time for event
    "final_time": 5.00,  # Final time for event
    "dt": 0.0005,  # timestep size
    "output_frequency": 200,  # how frequently to output solution to pvds
    "gradient_sampling_frequency": 1,  # how frequently to save solution to RAM
}
default_dictionary["visualization"] = {
    "forward_output": True,
    "forward_output_filename": "results/forward_output.pvd",
    "fwi_velocity_model_output": False,
    "velocity_model_filename": None,
    "gradient_output": False,
    "gradient_filename": "results/Gradient.pvd",
    "adjoint_output": False,
    "adjoint_filename": None,
    "debug_output": False,
}
optimization_parameters = {
    "General": {
        "Secant": {"Type": "Limited-Memory BFGS", "Maximum Storage": 10}
    },
    "Step": {
        "Type": "Augmented Lagrangian",
        "Augmented Lagrangian": {
            "Subproblem Step Type": "Line Search",
            "Subproblem Iteration Limit": 5.0,
        },
        "Line Search": {"Descent Method": {"Type": "Quasi-Newton Step"}},
    },
    "Status Test": {
        "Gradient Tolerance": 1e-16,
        "Iteration Limit": None,
        "Step Tolerance": 1.0e-16,
    },
}
default_dictionary["inversion"] = {
    "perform_fwi": False,  # switch to true to make a FWI
    "initial_guess_model_file": None,
    "shot_record_file": None,
    "optimization_parameters": optimization_parameters,
}

class Overthrust_acoustic(Example_model_acoustic):
    """
    Rectangle model.
    This class is a child of the Example_model class.
    It is used to create a dictionary with the parameters of the
    Rectangle model.

    Parameters
    ----------
    dictionary : dict, optional
        Dictionary with the parameters of the model that are different from
        the default model. The default is None.
    comm : firedrake.mpi_comm.MPI.Intracomm, optional
    periodic : bool, optional
        If True, the mesh will be periodic in all directions. The default is
        False.
    """
    def __init__(
        self,
        dictionary=None,
        example_dictionary=default_dictionary,
        comm=None,
        periodic=False,
    ):
        super().__init__(
            dictionary=dictionary,
            default_dictionary=example_dictionary,
            comm=comm,
        )

We can now create multiple wave objects just by varying the desired variables. The example below creates 2 different objects with different source locations.

In [9]:
temp_dict = {}
temp_dict["acquisition"] = {
    "source_locations": [(-0.01, 1.0)],
    "frequency": 5.0,
    "receiver_locations": spyro.create_transect((-0.37, 0.2), (-0.37, 5.8), 10),
}
Wave_obj_overthurst1 = Overthrust_acoustic(dictionary=temp_dict)
temp_dict["acquisition"]["source_locations"] = [(-0.01, 5.0)]
Wave_obj_overthurst2 = Overthrust_acoustic(dictionary=temp_dict)

Parallelism type: automatic
INFO: Distributing 1 shot(s) across 1 core(s).                 Each shot is using 1 cores
  rank 0 on ensemble 0 owns 5976 elements and can access 3113 vertices
Parallelism type: automatic
INFO: Distributing 1 shot(s) across 1 core(s).                 Each shot is using 1 cores
  rank 0 on ensemble 0 owns 5976 elements and can access 3113 vertices


This is also recommended for reused examples that require new methods not in the inherited class. 